In [2]:
import numpy as np
import pandas as pd
import import_ipynb
import glob

In [3]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

In [ ]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [4]:
import matplotlib.pyplot as plt 

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [5]:
import internal_inconsistencies as ii

importing Jupyter notebook from internal_inconsistencies.ipynb
importing Jupyter notebook from data_preperation.ipynb


In [ ]:
for abbr in abbr_list:    
    df = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv').iloc[:,2:]
    df = df.rename(columns={df.columns[0]: 'demand'})
    print(abbr)

    X = np.arange(len(df.index)).reshape(len(df.index), 1)
    y = df[column].tolist()

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    rmses = []
    degrees = np.arange(1, 10)
    min_rmse, min_deg = 1e10, 0

    for deg in degrees:

        # Train features
        poly_features = PolynomialFeatures(degree=deg, include_bias=False)
        x_poly_train = poly_features.fit_transform(x_train)

        # Linear regression
        poly_reg = LinearRegression()
        poly_reg.fit(x_poly_train, y_train)

        # Compare with test data
        x_poly_test = poly_features.fit_transform(x_test)
        poly_predict = poly_reg.predict(x_poly_test)
        poly_mse = mean_squared_error(y_test, poly_predict)
        poly_rmse = np.sqrt(poly_mse)
        rmses.append(poly_rmse)
        
        # Cross-validation of degree
        if min_rmse > poly_rmse:
            min_rmse = poly_rmse
            min_deg = deg

        poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
        x_poly_train = poly_features.fit_transform(x_train)
        poly_reg = LinearRegression()
        poly_reg.fit(x_poly_train, y_train)
        x_poly_test = poly_features.fit_transform(x_test)
        poly_predict = poly_reg.predict(x_poly_test)

    # Plot and present results
        print('{} Best degree {} with RMSE {}'.format(column,min_deg, min_rmse))
        
    # fig = plt.figure()
    # ax = fig.add_subplot(111)
    # ax.plot(degrees, rmses)
    # ax.set_yscale('log')
    # ax.set_xlabel('Degree')
    # ax.set_ylabel('RMSE')


In [7]:
def regression(df):
    edited_df = pd.DataFrame()
    df, divider = ii.omit_dst(df)
    df = df.replace('n/e', 0)
    for column in df.columns.values:
        if not (df[column] == 0).all():
            column_data = [[index, value] for index, value in enumerate(df.loc[:, column])]
            for index, value in column_data:
                if pd.isnull(value):
                    list_null.append(index)
                    temp_list = [item for item in column_data if (index-item[0]) % (24*divider) == 0]
                    temp_list.sort()
                    X = np.array([i[0] for i in temp_list[1:]]).reshape(len(temp_list)-1, 1)
                    y = [i[1] for i in temp_list[1:]]
                    y = [0 if pd.isna(x) else x for x in y]

                    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

                    degrees = np.arange(1, 10)
                    min_rmse, min_deg = 1e10, 0

                    for deg in degrees:

                            # Train features
                        poly_features = PolynomialFeatures(degree=deg, include_bias=False)
                        x_poly_train = poly_features.fit_transform(x_train)

                        # Linear regression
                        poly_reg = LinearRegression()
                        poly_reg.fit(x_poly_train, y_train)

                        # Compare with test data
                        x_poly_test = poly_features.fit_transform(x_test)
                        poly_predict = poly_reg.predict(x_poly_test)
                        poly_mse = mean_squared_error(y_test, poly_predict)
                        poly_rmse = np.sqrt(poly_mse)

                        # Cross-validation of degree
                        if min_rmse > poly_rmse:
                            min_rmse = poly_rmse
                            min_deg = deg

                    poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
                    x_poly_train = poly_features.fit_transform(x_train)
                    poly_reg = LinearRegression()
                    poly_reg.fit(x_poly_train, y_train)

                    df.loc[index,column] = poly_reg.predict(poly_features.fit_transform([[0]]))[0]
            edited_df = pd.concat([edited_df, df.loc[:, column]], axis=1) 
    edited_df[edited_df<0]=0
    return edited_df
df = pd.read_csv('../Data Sources/ENTSO-E/2018/Generation/Belgium.csv').iloc[:, 2:]
# df['Hydro Pumped Storage  - Actual Aggregated [MW]'] = df['Hydro Pumped Storage  - Actual Aggregated [MW]'].fillna(0)
df = df.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'], axis=1)



     Biomass  - Actual Aggregated [MW]  \
125                              371.0   
126                              368.0   
127                              370.0   

     Fossil Brown coal/Lignite  - Actual Aggregated [MW]  \
125                                                  0     
126                                                  0     
127                                                  0     

     Fossil Coal-derived gas  - Actual Aggregated [MW]  \
125                                                  0   
126                                                  0   
127                                                  0   

     Fossil Gas  - Actual Aggregated [MW]  \
125                                1656.0   
126                                1651.0   
127                                1623.0   

     Fossil Hard coal  - Actual Aggregated [MW]  \
125                                         0.0   
126                                         0.0   
127                    

In [29]:
print(poly_predict)

[7727.41661229 5988.4734279  6367.59351269 ... 7017.13446151 6982.32233727
 6083.31417277]


In [ ]:
df = pd.read_csv('../Data Sources/Output/Nearest Neighbours Mean/Load/FR.csv')
x = df.index.values
y = df['demand'].tolist()
plt.figure(figsize=(30, 10))
plt.plot(x, y)
# for csv in csvs:
#     df = pd.read_csv(csv).iloc[:,1:]
#     print(csv)
#     df, length = ii.omit_dst(df)
#     df = df.replace('n/e', 0)
#     for column in df.columns.values:
#         if not (df[column] == 0).all():
#             column_data = [[index, value] for index, value in enumerate(df.loc[:, column])]
#             for index, value in column_data:
#                 if pd.isnull(value):
#                     temp_list = [[abs(index-item[0]),item[1]] for item in column_data if (index-item[0]) % (24*length) == 0]
#                     temp_list.sort()
#                     nearest_neighbours = [x[1] for x in temp_list[:60]]
#                     nearest_neighbours=[0 if pd.isna(x) else x for x in nearest_neighbours]
#                     # print(index,nearest_neighbours)
#                     # print(np.mean(np.array(nearest_neighbours)))
#                     df.loc[index, column] = np.mean(np.array(nearest_neighbours))
#             transmission_consolidated_nnm = pd.concat([transmission_consolidated_nnm, df.loc[:, column]], axis=1)
# transmission_consolidated_nnm


In [ ]:
X = np.arange(len(df.index)).reshape(len(df.index), 1)
X


In [ ]:
x = [[1,2],[2,3],[3,4]]
for i,j in x:
    print(i,j)